# Laboratorium Automatyka Pojazdowa: system wizyjny rozpoznający znaki drogowe

## Część 2: Lokalizacja

Problem do rozwiązania w ramach tego laboratorium jest zadaniem lokalizacji: określenia regionów, w których znajdują się na obrazie obiekty - w naszym przypadku, znaki drogowe, ale jeszcze bez wskazania klasy.
W rzeczywistości wskazany problem jest skomplikowany, zatem wykorzystany zostanie transfer learning - technika polegająca na wykorzystaniu wytrenowanego na dużo bardziej zaawansowanym, szerszym, datasecie, modelu ekstraktora cech (sieci konwolucyjnej) i trenowanie jedynie wyjściowej warstwy gęsto połączonej (tzw. backbone-a).

Wykorzystane zostanie Tensorflow Object Detection API oraz [model RetinaNet](https://paperswithcode.com/method/retinanet) pretrenowany na datasecie [MS COCO (Common Objects in COntext)](https://cocodataset.org/), który stanowi bardzo dobry detektor (czyli model przeprowadzający lokalizację oraz klasyfikację jednocześnie) obiektów.
Wykorzystywany dataset posiada jedynie informacje o bounding boxach oznaczających znaki w katalogu [data/detection/labels](data/detection/labels) (wystarczy spojrzeć na dowolny plik TXT z labelem, np. [`1487.txt`](../data/detection/labels/1487.txt) oraz dla "orientacji" [`1487.jpg`](../data/detection/imgs/1487.jpg)) - w każdym pliku każda linia jest postaci `<klasa> <xCenter %> <yCenter %> <width %> <height %>`, jednakże `klasa` zawsze jest 0 - dataset nie zawiera więcej informacji. Z uwagi na to, w tym ćwiczeniu nie tworzymy w praktyce detektora, a jedynie lokalizator.

Miejsca w kodzie pozostawione do uzupełnienia zostały w tym notatniku oznaczone `# TODO`.

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from typing import *
import os
import cv2

import tensorflow_models as tfm

from official.core import exp_factory
from official.core import config_definitions as cfg
from official.vision.serving import export_saved_model_lib
from official.vision.ops.preprocess_ops import normalize_image
from official.vision.ops.preprocess_ops import resize_and_crop_image
from official.vision.utils.object_detection import visualization_utils
from official.vision.dataloaders.tf_example_decoder import TfExampleDecoder

%matplotlib inline

## Konwersja datasetu do formatu TFrecord

[TF Models API - TFM]() - operuje na datasetach w postaci TFrecord. Jest to format skompresowanych i zoptymalizowanych zbiorów danych dla tensorflow, zapisywany do jednego pliku.
W tej sekcji zadaniem dla Państwa jest:
- zaczytanie zdjęć oraz plików TXT w formacie opisanym w sekcji powyżej w celu utworzenia zbioru danych (`tf.data.Dataset`) z obrazami oraz oznaczonymi na nich bounding boxami jedynej klasy - znaku drogowego ('jakiegokolwiek')
- podziału datasetu na zbiór treningowy i ewaluacyjny w stosunku $80 : 20$
- zapisania obydwóch datasetów do plików: `signs-train.tfrecord` oraz `signs-eval.tfrecord`

#### Wczytywanie danych

W poniższej komórce znajdują się parametry odpowiedzialne za konstrukcję datasetu, dla których - po wypełnieniu pozostałych komórek - kolejno należy uruchamiać notatnik i zanotować wyniki: `BATCH_SIZE` - należy sprawdzić kolejno działanie dla wartości 4, 16, 32. Batch size - rozmiar wsadu - to parametr sterujący ilością obrazków w pojedynczym wsadzie, dla którego są kolejno wykonywane: forward propagation, backward propagation, optymalizacja wag sieci.

In [ ]:
BATCH_SIZE = ... # TODO: należy dobrać sensowny rozmiar wsadu
IMG_W = ... # TODO: ustawić na 512px
IMG_H = ... # TODO: ustawić na 256px

W poniższej komórce znajdują się implementacje metod odpowiedzialnych za ładowanie danych.

In [ ]:
def readImage(imagePath: str):
    """Funkcja ładująca obrazek ze ścieżki imagePath"""
    rawBytes = tf.io.read_file(imagePath)
    
    return tf.io.decode_jpeg(rawBytes)

def read_image_size_for_tf(image: tf.Tensor) -> List[int]:
    # TODO: image jest tensorem tf.Tensor; aby otrzymać np.ndarray będący obrazem RGB o kształcie [Height, Width, Depth], należy wykonać image.numpy()
    # poniższy kod ma zwracać Listę 2 integerów w postaci [Width, Height]
    ...
    return [W, H]

# TODO: poniżej należy wskazać relatywną ścieżkę do katalogu detection/imgs w datasecie
images = tf.data.Dataset.list_files('../<fragment sciezki - TODO>/*.jpg', shuffle=False)
images = images.map(readImage)
imageSizesOriginal = images.map(lambda x: tf.py_function(read_image_size_for_tf, [x], [tf.uint64, tf.uint64]))
images = images.map(lambda x: tf.image.resize(x, (IMG_H, IMG_W)))
# TODO: znormalizuj wartości pikseli (z obecnych elementów uint8 0-255 na float32 0-1)
images = images.map(lambda image: ...)

def load_labels(label_path):
    BBs = []
    label_path = label_path.numpy()

    with open(label_path) as f:
        for line in filter(lambda l: len(l.strip()), f.readlines()):
            _, xCenter, yCenter, width, height = map(float, line.split())

            # TODO: wylicz współrzędne górnego lewego punktu bounding boxa (xMin, yMin) oraz prawego dolnego (xMax, yMax)
            halfSignW = ...
            halfSignH = ...
            xMin = ...
            yMin = ...
            xMax = ...
            yMax = ...

            BBs.append((yMin, xMin, yMax, xMax))

    filename = os.path.basename(label_path)
    return filename, BBs

# TODO: poniżej należy wskazać relatywną ścieżkę do katalogu detection/labels w datasecie
labelPaths = tf.data.Dataset.list_files('../<fragment sciezki - TODO>/*.txt', shuffle=False)
labels = labelPaths.map(lambda labelPath: tf.py_function(load_labels, [labelPath], [tf.string, tf.float32]))
dataset = tf.data.Dataset.zip(images, labels)

dataset = dataset.shuffle(int(len(dataset) / 8)) # shuffle the data a bit (1/8-perfect shuffling)

# TODO: poniżej należy wyliczyć rozmiar datasetu testowego tak, by był w stosunku do rozmiaru datasetu treningowego jak 2:8
# podpowiedź: należy skorzystać z tego, że tf.data.Dataset implementuje metodę __len__(), zatem można na nim użyć len(...)
TEST_SIZE: int = int(...)

test_dataset: tf.data.Dataset = dataset.take(TEST_SIZE) 
train_dataset: tf.data.Dataset = dataset.skip(TEST_SIZE)

print(f"Loaded {len(dataset)} dataset samples")

print()
print("1st sample:")
samplesIter = iter(dataset)
sample = next(samplesIter)
(sampleImage, sampleLabel) = sample
print("Sample type:", type(sample))
print("Sample image shape:", sampleImage.numpy().shape)
print("Sample label:")
print(sampleLabel)

In [20]:
test_dataset.save("../data/localization-test-dataset")

### Wizualizacja danych

In [ ]:
def drawBB(image: np.ndarray, text: str, color: Tuple[int, int, int], BB: Tuple[int, int, int, int], scale: float):
    x, y, w, h = BB

    cv2.rectangle(
        image,
        # TODO: poniżej:
        # - podać jako pt1 współrzędne lewego górnego rogu BB
        # - podać jako pt2 współrzędne prawego dolnego rogu BB
        pt1 = ...,
        pt2 = ...,
        color=color,
        thickness=int(10 * scale)
    )
    cv2.putText(
        image,
        text=text,
        org=(x, y - 40),
        fontFace=cv2.FONT_HERSHEY_COMPLEX,
        fontScale=1.3 * scale,
        color=(255, 255, 255),
        thickness=int(6 * scale)
    )

In [ ]:
PREVIEW_COLS = 3
PREVIEW_ROWS = 3
FIG_UNIT_SIZE_W, FIG_UNIT_SIZE_H = 4, 2

fig, axs = plt.subplots(nrows=PREVIEW_ROWS, ncols=PREVIEW_COLS, figsize=(PREVIEW_COLS * FIG_UNIT_SIZE_W, PREVIEW_ROWS * FIG_UNIT_SIZE_H))
print(f"Displaying {PREVIEW_COLS * PREVIEW_ROWS} random train dataset samples")
fig.tight_layout()

# create an iterator to a copy of the dataset unbatched, i.e., iterated not in batches of shape (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, 3), but (IMAGE_SIZE, IMAGE_SIZE, 3) - single RGB images
samplesIter = iter(train_dataset)

for i in range(PREVIEW_ROWS):
    for j in range(PREVIEW_COLS):
        image, (filename, BBs) = next(samplesIter)
        image = image.numpy().reshape(1, IMG_H, IMG_W, 3)[0] # tf.Tensor -> np.ndarray, np.ndarray as uint8 ([0, 255])

        for bbi, BB in enumerate(BBs):
            (yMin, xMin, yMax, xMax) = BB
            
            # TODO: poniżej przeliczyć współrzędne wyrażone w wartości znormalizowanej (yMin, xMin, yMax, xMax) na bezwzględne współrzędne pikselowe
            xMin = int(...)
            xMax = int(...)
            yMin = int(...)
            yMax = int(...)

            drawBB(
                image,
                text=f"",
                color=(0, 1, 0),
                BB=(
                    # TODO: podać poniżej - kolejno - wartości:
                    # - X lewego górnego rogu
                    # - Y lewego górnego rogu
                    # - szerokości BB
                    # - wysokości BB
                    ...,
                    ...,
                    ...,
                    ...,
                ),
                scale=0.5
            )

        axs[i][j].imshow(image, vmin=0, vmax=1)
        axs[i][j].set_title(f"Sample {(i + 1) * PREVIEW_ROWS + j} (detections: {len(BBs)})")
        axs[i][j].axis("off")

plt.show()

### Konwersja przykładów do `tf.train.Example`s i zapis do plików `.tfrecord`

In [ ]:
signClassAsBytes = "Sign".encode('utf8')
jpegStrAsBytes = "jpg".encode('utf8')

def create_tf_example(BBs, image, filename: str) -> str:
    ymins = [BB[0] for BB in BBs]
    xmins = [BB[1] for BB in BBs]
    ymaxs = [BB[2] for BB in BBs]
    xmaxs = [BB[3] for BB in BBs]
    classes_text = [signClassAsBytes] * len(BBs)
    classes = [1] * len(BBs)
    filename = os.path.splitext(filename)[0].encode("utf-8")

    image = (image * 255.0).astype(np.uint8)
    imageEncoded = tf.image.encode_jpeg(image, format='rgb').numpy()

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': tf.train.Feature(int64_list=tf.train.Int64List(value=[IMG_H])),
        'image/width': tf.train.Feature(int64_list=tf.train.Int64List(value=[IMG_W])),
        'image/filename': tf.train.Feature(bytes_list=tf.train.BytesList(value=[filename])),
        'image/source_id': tf.train.Feature(bytes_list=tf.train.BytesList(value=[filename])),
        'image/encoded': tf.train.Feature(bytes_list=tf.train.BytesList(value=[imageEncoded])),
        'image/format': tf.train.Feature(bytes_list=tf.train.BytesList(value=[jpegStrAsBytes])),
        'image/object/bbox/xmin': tf.train.Feature(float_list=tf.train.FloatList(value=xmins)),
        'image/object/bbox/xmax': tf.train.Feature(float_list=tf.train.FloatList(value=xmaxs)),
        'image/object/bbox/ymin': tf.train.Feature(float_list=tf.train.FloatList(value=ymins)),
        'image/object/bbox/ymax': tf.train.Feature(float_list=tf.train.FloatList(value=ymaxs)),
        'image/object/class/text': tf.train.Feature(bytes_list=tf.train.BytesList(value=classes_text)),
        'image/object/class/label': tf.train.Feature(int64_list=tf.train.Int64List(value=classes)),
    }))

    return tf_example.SerializeToString()

for ds, datasetLabel in [
   (train_dataset, "train"),
   (test_dataset, "eval")
]:
    outName = f"signs-{datasetLabel}.tfrecord"
    writer = tf.io.TFRecordWriter(outName)

    print(f"Writing {len(ds)} samples to {outName} (dataset '{datasetLabel}')")
    
    for image, (filename, label) in ds:
        filename = filename.numpy()

        writer.write(
            create_tf_example(
                BBs=label,
                image=image.numpy(),
                filename=filename.decode()
            )
        )
    writer.close()

## Fine-tuning sieci

#### Konfiguracja zadania trenowania

Należy najpierw skonfigurować zadanie trenowania TFM (TF Models API), które spowoduje pobranie i rozpakowanie modelu, który będzie fine-tune'owany (tzn. warstwa wyjściowa będzie trenowana ponownie).

Ponadto, po każdych 100 epokach trenowania, checkpoint modelu będzie zapisywany na dysku i będzie uruchamiana ewaluacja, która wyliczy wartości metryk [według specyfikacji ewaluacji datasetu COCO](https://cocodataset.org/#detection-eval) (**z których opisami należy się zapoznać!**) dla modelu w stanie na danym etapie trenowania.

In [ ]:
batch_size = 16
num_classes = 1 # tylko 1 klasa: znak drogowy

train_data_input_path = './signs-train.tfrecord'
valid_data_input_path = './signs-eval.tfrecord'
test_data_input_path = './signs-eval.tfrecord'
model_dir = './trained_model/'
export_dir ='../models/localizer'

exp_config = exp_factory.get_exp_config('retinanet_resnetfpn_coco') # fasterrcnn_resnetfpn_coco retinanet_resnetfpn_coco

IMG_SIZE = [IMG_H, IMG_W, 3]

# Backbone config.
exp_config.task.freeze_backbone = True # do not train the backbone - we just fine-tune the last layer
exp_config.task.annotation_file = ''

# Model config.
exp_config.task.model.input_size = IMG_SIZE
exp_config.task.model.num_classes = num_classes + 1
# exp_config.task.model.detection_generator.tflite_post_processing.max_classes_per_detection = exp_config.task.model.num_classes

# Training data config.
exp_config.task.train_data.input_path = train_data_input_path
exp_config.task.train_data.dtype = 'float32'
exp_config.task.train_data.global_batch_size = batch_size
exp_config.task.train_data.parser.aug_scale_max = 1.0
exp_config.task.train_data.parser.aug_scale_min = 1.0

# Validation data config.
exp_config.task.validation_data.input_path = valid_data_input_path
exp_config.task.validation_data.dtype = 'float32'
exp_config.task.validation_data.global_batch_size = batch_size

Następnie konfigurowana jest ilość epok (iteracji trenowania) oraz ilość kroków na epokę. Po każdej epoce przeprowadzana będzie walidacja - sprawdzenie metryk sieci na zbiorze ewaluacyjnym.

In [ ]:
train_steps = 7500
exp_config.trainer.steps_per_loop = 100 # steps_per_loop = num_of_training_examples // train_batch_size

exp_config.trainer.summary_interval = 100
exp_config.trainer.checkpoint_interval = 100
exp_config.trainer.validation_interval = 100
exp_config.trainer.validation_steps =  100 # validation_steps = num_of_validation_examples // eval_batch_size
exp_config.trainer.train_steps = train_steps
exp_config.trainer.optimizer_config.warmup.linear.warmup_steps = 100
exp_config.trainer.optimizer_config.learning_rate.type = 'cosine'
exp_config.trainer.optimizer_config.learning_rate.cosine.decay_steps = train_steps
exp_config.trainer.optimizer_config.learning_rate.cosine.initial_learning_rate = 0.1
exp_config.trainer.optimizer_config.warmup.linear.warmup_learning_rate = 0.05

# pprint.PrettyPrinter(indent=4).pprint(exp_config.as_dict())

distribution_strategy = tf.distribute.MirroredStrategy()
with distribution_strategy.scope():
  task = tfm.core.task_factory.get_task(exp_config.task, logging_dir=model_dir)

for images, labels in task.build_inputs(exp_config.task.train_data).take(1):
  print()
  print(f'images.shape: {str(images.shape):16}  images.dtype: {images.dtype!r}')
  print(f'labels.keys: {labels.keys()}')

#### Konfiguracja mapy etykiet do ID klas

In [ ]:
category_index = {
    # TODO: wpisać do dicta category_index klucz 1 o wartości będącej dictem z kluczami:
    # - id o wartości 1
    # - name o wartości "Sign"
}

#### Wizualizacja przykładowych danych z datasetu treningowego wraz z bounding boxami

In [ ]:
%matplotlib inline
tf_ex_decoder = TfExampleDecoder()

def visualizeBatchOfRecords(recordsRaw, count):
    plt.figure(figsize=(20, 20))

    for i, serialized_example in enumerate(recordsRaw):
        plt.subplot(1, count, i + 1)
        loadedTensors = tf_ex_decoder.decode(serialized_example)
        groundTruthBoxes = loadedTensors['groundtruth_boxes'].numpy()
        image = loadedTensors['image'].numpy()
        scores = np.ones(shape=(len(groundTruthBoxes)))

        visualization_utils.visualize_boxes_and_labels_on_image_array(
            image,
            groundTruthBoxes,
            loadedTensors['groundtruth_classes'].numpy().astype('int'),
            scores,
            category_index=category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=200,
            min_score_thresh=0.5,
            agnostic_mode=False,
            instance_masks=None,
            line_thickness=4)

        plt.imshow(image)
        plt.axis('off')
        plt.title(f'Image-{i + 1}')
    
    plt.show()

visualizeBatchOfRecords(tf.data.TFRecordDataset(exp_config.task.train_data.input_path).take(4).shuffle(buffer_size=2), 4)

#### Uruchomienie fine-tuningu

Wpisana wyżej konfiguracja i dane zostaną teraz wykorzystane do uruchomienia procesu fine-tuningu.

**Uwaga:** ten proces potrwa ~10 minut (tylko z uwagi na bardzo mocny sprzęt w tej sali - jest to relatywnie "mało"). Na bieżąco będzie można przeglądać wykresy metryk modelu aktualizowane podczas kroku eval (ewaluacji) w lokalnie uruchomionym Tensorboard ([localhost:6006](http://localhost:6006)) uruchamianym w komórce poniżej.

In [ ]:
%load_ext tensorboard
!sleep 8
%tensorboard --logdir './trained_model/'

In [ ]:
model, eval_logs = tfm.core.train_lib.run_experiment(
    distribution_strategy=distribution_strategy,
    task=task,
    mode='train_and_eval',
    params=exp_config,
    model_dir=model_dir,
    run_post_eval=True
)

In [19]:
# wyeksportowanie grafu inferencji - zoptymalizowanego formatu modelu wraz z wyuczonymi wagami, na którym można efektywnie przeprowadzać inferencję - tj. predykcję dla dowolnych danych wejściowych

export_saved_model_lib.export_inference_graph(
    input_type='image_tensor',
    batch_size=1,
    input_image_size=[IMG_H, IMG_W],
    params=exp_config,
    checkpoint_path=tf.train.latest_checkpoint(model_dir),
    export_dir=export_dir
)

INFO:tensorflow:Assets written to: ../models/localizer/assets


INFO:tensorflow:Assets written to: ../models/localizer/assets


## Uruchomienie inferencji na załadowanym zamrożonym modelu sieci

### 3 przykłady z datasetu - wraz z 'ground truth' bounding boxes

In [ ]:
count = 3

evalDataset = tf.data.TFRecordDataset(
    './signs-eval.tfrecord').take(
        count)
visualizeBatchOfRecords(evalDataset, count)
imported = tf.saved_model.load(export_dir)
modelFunction = imported.signatures['serving_default']

IMG_SIZE = (IMG_H, IMG_W)
plt.figure(figsize=(20, 20))

### 3 przykłady z datasetu - wraz z bounding boxami pochodzącymi z inferencji na sieci neuronowej

In [ ]:
plt.figure(figsize=(20,5))

for i, example in enumerate(evalDataset):
  plt.subplot(1, 3, i + 1)
  loadedTensors = tf_ex_decoder.decode(example)
  imagePath = resize_and_crop_image(
      loadedTensors['image'],
      IMG_SIZE,
      padded_size=IMG_SIZE,
      aug_scale_min=1.0,
      aug_scale_max=1.0
  )[0]
  imagePath = tf.expand_dims(imagePath, axis=0)
  imagePath = tf.cast(imagePath, dtype = tf.uint8)
  image_np = imagePath[0].numpy()
  result = modelFunction(imagePath)
  visualization_utils.visualize_boxes_and_labels_on_image_array(
      image_np,
      result['detection_boxes'][0].numpy(),
      result['detection_classes'][0].numpy().astype(int),
      result['detection_scores'][0].numpy(),
      category_index=category_index,
      use_normalized_coordinates=False,
      max_boxes_to_draw=200,
      min_score_thresh=..., # TODO: należy ustawić tą wartość na rozsądny próg (znormalizowany - zakres wartości to 0-1)
      min_score_thresh=0.6,
      agnostic_mode=False,
      instance_masks=None,
      line_thickness=4)
  plt.imshow(image_np)
  plt.axis('off')

plt.show()

## Wyniki

### Instrukcja

Jako podsumowanie laboratorium należy:
- zapisać jako plik `img/localizedBBs.png` powyższą grafikę ze zlokalizowanymi znakami na 3 losowych obrazach z ewaluacyjnego zbioru danych
- zapisać fragment ekranu Tensorboard z odpowiednim wykresem jako plik `img/tensorboard-AP.png`
- zapisać fragment ekranu Tensorboard z odpowiednim wykresem jako plik `img/tensorboard-AP50.png`
- zapisać fragment ekranu Tensorboard z odpowiednim wykresem jako plik `img/tensorboard-AP75.png`
- zapoznać się z [dokumentacją standardów ewaluacji datasetu COCO](https://cocodataset.org/#detection-eval) i opisać:
    - czym są metryki:
        - AP
        - AP50
        - AP75
    - jak należy interpretować wyniki AP, AP50 i AP75 w naszym przypadku: należy opisać, jakie wartości metryk zostały obliczone dla naszego modelu oraz co to oznacza w praktyce?

### Przykładowa lokalizacja

<img src="img/localizedBBs.png?q=1" />

Komentarz dot. skuteczności: ...

### Metryki

| Metryka | Wartość | Opis metryki | Interpretacja uzyskanej wartości |       Screenshot wykresu z Tensorboard       |
|---------|---------|--------------|----------------------------------|----------------------------------------------|
| AP      |         |              |                                  |  <img src="img/tensorboard-AP.png?q=1" />    |
| AP50    |         |              |                                  |  <img src="img/tensorboard-AP50.png?q=1" />  |
| AP75    |         |              |                                  |  <img src="img/tensorboard-AP75.png?q=1" />  |

### Interpretacja wyników

...